#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [3]:
def entropy_func(class_count, num_samples):
    entropy = 0
    for count in class_count.values():
        probability = count / num_samples
        entropy -= probability * math.log2(probability)
    return entropy


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

    def group_entropy(self):
        class_count = Counter(self.group_classes)
        return entropy_func(class_count, len(self.group_classes))


class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if self.val is not None:
            return self.val
        if data[self.split_feature] <= self.split_val:
            return self.child_node_a.predict(data)
        else:
            return self.child_node_b.predict(data)


class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a, group_b):
        total_samples = len(group_a) + len(group_b)
        total_entropy = (len(group_a) / total_samples) * group_a.entropy + (len(group_b) / total_samples) * group_b.entropy
        return total_entropy

    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        return parent_group.entropy - self.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_values, classes):
        best_gain = 0
        best_feature = None
        best_split_val = None
        parent_group = Group(classes)

        for feature in range(len(feature_values[0])):
            chosen_feature = feature_values[:, feature]
            unique_values = np.unique(chosen_feature)

            for val in unique_values:
                temp_a_indexes = []
                temp_b_indexes = []
                for idx in range(len(chosen_feature)):
                    if chosen_feature[idx] <= val:
                        temp_a_indexes.append(idx)
                    else:
                        temp_b_indexes.append(idx)
                group_a_indexes = np.asarray(temp_a_indexes, dtype=int)
                group_b_indexes = np.asarray(temp_b_indexes, dtype=int)

                group_a = Group(classes[group_a_indexes])
                group_b = Group(classes[group_b_indexes])


                gain = self.get_information_gain(parent_group, group_a, group_b)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_split_val = val

        return best_feature, best_split_val

    def get_best_split(self, data, classes):
        # zwrot najczęściej występującej klasy
        if self.depth >= self.max_depth or len(np.unique(classes)) == 1:
            return Node(split_feature=None, split_val=None, val=Counter(classes).most_common(1)[0][0])

        best_feature, best_split_val = self.get_best_feature_split(data, classes)

        temp_a_indexes = []
        temp_b_indexes = []
        for idx in range(len(data)):

            if data[idx, best_feature] <= best_split_val:
                temp_a_indexes.append(idx)
            else:
                temp_b_indexes.append(idx)
        group_a_indexes = np.asarray(temp_a_indexes, dtype=int)
        group_b_indexes = np.asarray(temp_b_indexes, dtype=int)

        self.depth += 1
        left_node = self.get_best_split(data[group_a_indexes], classes[group_a_indexes])
        right_node = self.get_best_split(data[group_b_indexes], classes[group_b_indexes])

        return Node(best_feature, best_split_val, self.depth, left_node, right_node)

    def build_tree(self, data, classes):
        self.tree = self.get_best_split(data, classes)

    def predict(self, data):
        return self.tree.predict(data)

In [4]:
# dla małej głębokości
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123+i)


    dc = DecisionTreeClassifier(1)
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')


Test:1; Predykcja:1
Test:2; Predykcja:1
Test:2; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:1
Test:2; Predykcja:1
effectiveness: 0.6
Test:1; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:2
Test:1; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:2
Test:1; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.5333333333333333
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:2; Predykcja:1
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:1
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:2; Predykcja:1
Test:1; Predykcja:1
effectiveness: 0.6
Test:0; Predykcja:0
Test:0; 

In [5]:
# dla najlepszej głębokości
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123+i)


    dc = DecisionTreeClassifier(3)
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')

Test:1; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.9333333333333333
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.8666666666666667
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
effectiveness: 0.933333333333333

In [17]:
# mały zbiór weryfikujący
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.95, random_state=123+i)


    dc = DecisionTreeClassifier(3)
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')

Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:1
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:1


In [11]:
# mały zbiór uczący
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.01, random_state=123+i)


    dc = DecisionTreeClassifier(3)
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')

Test:1; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.5
Test:1; Predykcja:1
Test:0; Predykcja:0
effectiveness: 1.0
Test:0; Predykcja:0
Test:2; Predykcja:2
effectiveness: 1.0
Test:0; Predykcja:0
Test:0; Predykcja:0
effectiveness: 1.0
Test:2; Predykcja:2
Test:1; Predykcja:1
effectiveness: 1.0
Test:0; Predykcja:0
Test:1; Predykcja:1
effectiveness: 1.0
Test:0; Predykcja:0
Test:1; Predykcja:1
effectiveness: 1.0
Test:1; Predykcja:1
Test:2; Predykcja:2
effectiveness: 1.0
Test:0; Predykcja:0
Test:1; Predykcja:1
effectiveness: 1.0
Test:1; Predykcja:1
Test:1; Predykcja:1
effectiveness: 1.0
Average accuracy: 0.95


In [16]:
# duża głębokość
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123+i)


    dc = DecisionTreeClassifier(20)
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')

Test:1; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.8666666666666667
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:2
Test:2; Predykcja:2
effectiveness: 0.8666666666666667
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
effectiveness: 0.933333333333333

In [21]:
# podział zbiorów na pół
iris = load_iris()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=123+i)


    dc = DecisionTreeClassifier(5) # 3 najlepsze
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')

Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:1
Test:0; Predykcja:1
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:0; Predykcja:0


wnioski: wraz z głębokością drzewa zwiększa się jego zdolność do poprawnej predykcji klasy, jednak tylko do pewnego momentu. Przy dużych głębokościach algorytm nie poprawia swojego działania, a nawet jest trochę gorszy. Wynika to z przekombinowania zadania, a jak wiemy najprostsze rozwiązania są najlepsze. Należy też zadbać o odpowiedni podział danych na częśc uczącą i weryfikującą. W przypadku uczącej muszą się w niej znajdować przedstawiciele wszystkich klas, gdy zbió uczący będzie zbyt mały drzewo nie będzie w stanie rozpoznać niektórych klas. W przypadku zbioru weryfikującego powinniśmy zadbać aby jego rozmiar umożliwiał dogłębne przetestowanie drzewa.
Mniejsza liczba poziomów drzewa oznacza mniejszą liczbę decyzji do podjęcia, a więc mniejsze ryzyko że niektóre z nich będą niewłaściwe. Mniejsza głębokość drzewa redukuje także ryzyko nadmiernego dopasowania. 
Dokonując podziałów preferujemy te, które prowadzą do zmniejszenia niejednorodności (nieczystości) klas po podziale, ponieważ przybliża nas to do kryterium stopu (jednolite klasy). Entropia przyjmuje wartości minimalne dla przy pełnej dominacji jednej klasy oraz wartości maksymalne przy jednostajnym rozkłądzie klas, dlatego przy wyborze podziału preferujemy mniejsze wartości (w kodzie wyliczamy zysk, czyli różnicę entropii po podziale dlatego bierzemy większą wartość).

In [8]:
from sklearn.datasets import load_iris

iris = load_iris()

print("Features (data):")
print(iris.data)
print(len(iris.data))

print("\nTarget (labels):")
print(iris.target)

print("\nFeature Names:")
print(iris.feature_names)

print("\nTarget Names:")
print(iris.target_names)

Features (data):
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 

In [40]:
from sklearn.datasets import load_wine

iris = load_wine()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123+i)


    dc = DecisionTreeClassifier(7) # 7 najlepsze
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')


Test:2; Predykcja:2
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:1
effectiveness: 1.0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2; Predykcja:1
Test:2; Predykcja:2
Test:1; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:1; Predykcja:1
Test:1; Predykcja:0
Test:2; Predykcja:2
effectiveness: 0.8333333333333334
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:1; Predykcja:1
Test:2;

In [26]:
from sklearn.datasets import load_wine

iris = load_wine()

print("Features (data):")
print(iris.data)
print(len(iris.data))

print("\nTarget (labels):")
print(iris.target)

print("\nFeature Names:")
print(iris.feature_names)

print("\nTarget Names:")
print(iris.target_names)

Features (data):
[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]
178

Target (labels):
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

Feature Names:
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/o

In [25]:
from sklearn.datasets import load_digits

iris = load_digits()

print("Features (data):")
print(iris.data)
print(len(iris.data))

print("\nTarget (labels):")
print(iris.target)

print("\nFeature Names:")
print(iris.feature_names)

print("\nTarget Names:")
print(iris.target_names)

Features (data):
[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
1797

Target (labels):
[0 1 2 ... 8 9 8]

Feature Names:
['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_0', 'pixel_1_1', 'pixel_1_2', 'pixel_1_3', 'pixel_1_4', 'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_0', 'pixel_2_1', 'pixel_2_2', 'pixel_2_3', 'pixel_2_4', 'pixel_2_5', 'pixel_2_6', 'pixel_2_7', 'pixel_3_0', 'pixel_3_1', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5', 'pixel_3_6', 'pixel_3_7', 'pixel_4_0', 'pixel_4_1', 'pixel_4_2', 'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_4_6', 'pixel_4_7', 'pixel_5_0', 'pixel_5_1', 'pixel_5_2', 'pixel_5_3', 'pixel_5_4', 'pixel_5_5', 'pixel_5_6', 'pixel_5_7', 'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_3', 'pixel_6_4', 'pixel_6_5', 'pixel_6_6', 'pixel_6_7', 'pixe

In [52]:
from sklearn.datasets import load_digits

iris = load_digits()

x = iris.data
y = iris.target
sum_effect = 0
for i in range(0, 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123+i)


    dc = DecisionTreeClassifier(150) # 150 najlepsze
    dc.build_tree(x_train, y_train)
    predictions = []

    for sample, gt in zip(x_test, y_test):
        prediction = dc.predict(sample)
        predictions.append(prediction)
        print(f'Test:{gt}; Predykcja:{prediction}')
    effectiveness = np.mean(predictions == y_test)
    sum_effect += effectiveness
    print(f'effectiveness: {effectiveness}')
    predictions = []
print(f'Average accuracy: {sum_effect/10}')


Test:3; Predykcja:3
Test:3; Predykcja:3
Test:4; Predykcja:4
Test:4; Predykcja:4
Test:1; Predykcja:1
Test:3; Predykcja:3
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:7; Predykcja:3
Test:4; Predykcja:4
Test:0; Predykcja:0
Test:0; Predykcja:8
Test:5; Predykcja:5
Test:1; Predykcja:1
Test:6; Predykcja:6
Test:0; Predykcja:0
Test:3; Predykcja:3
Test:1; Predykcja:1
Test:0; Predykcja:0
Test:6; Predykcja:4
Test:9; Predykcja:9
Test:7; Predykcja:7
Test:7; Predykcja:7
Test:5; Predykcja:5
Test:4; Predykcja:4
Test:5; Predykcja:4
Test:7; Predykcja:7
Test:0; Predykcja:0
Test:0; Predykcja:0
Test:9; Predykcja:9
Test:1; Predykcja:1
Test:6; Predykcja:6
Test:8; Predykcja:8
Test:6; Predykcja:6
Test:8; Predykcja:1
Test:4; Predykcja:4
Test:6; Predykcja:6
Test:2; Predykcja:2
Test:1; Predykcja:8
Test:6; Predykcja:6
Test:0; Predykcja:0
Test:6; Predykcja:6
Test:2; Predykcja:2
Test:0; Predykcja:0
Test:2; Predykcja:2
Test:3; Predykcja:3
Test:5; Predykcja:4
Test:8; Predykcja:8
Test:8; Predykcja:8
Test:8; Predykcja:7
